# Single-state methods and Genetic Algorithms

- Random Search
- Hill Climbing
- Simulated Annealing
- Genetic Algorithms

Let us start by importing the random module (for ```random.choice```, ```random.choices```, and ```random.randint```) and the math module (for ```math.e```).

In [1]:
import random
import math

We define the **OneMax** function as the number of ones in a list

In [2]:
def onemax(xs):
  "The OneMax function. It returns the number of ones in a list"
  return len([x for x in xs if x == 1])

In [3]:
onemax([1,0,1,1,0])

3

## Random Search

For random search we generate ```n_iter``` times a new random solution that is accepted only if its fitness is equal or better than the best solution found so far

In [4]:
def random_search(fit, random_sample, n_iter = 100):
  best = random_sample()
  best_fit = fit(best)
  for i in range(0, n_iter):
    x = random_sample()
    x_fit = fit(x)
    if x_fit >= best_fit:
      best = x
      best_fit = x_fit
  return best

Let us try with OneMax and different values of $k$

In [5]:
random_sample = lambda: random.choices([0,1], k=20)
random_search(onemax, random_sample)

[1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1]

## Hill Climbing

Before defining Hill Climbing we will define some utility functions that will be employed in defining the neighbourhood of a given solution. In particular, how to move from and to a number and its representation as a list of numbers in binary.

In [6]:
def as_binary_string(k, n):
  xs = [0] * n
  i = 0
  while k != 0:
    if k % 2 == 1:
      xs[n-i-1] = 1
    k = k // 2
    i += 1
  return xs

def to_binary(xs):
  k = 0
  n = len(xs)
  for i, b in enumerate(xs):
    k += 2**(n-i-1) * b
  return k

In [7]:
to_binary([0,1,0,1,1])

11

Selecting one neightbour randomly when we consider a binary string as a number $k$ represented in binary is simply a choice between $k-1$ and $k+1$ represented as binary strings:

In [8]:
def neigh_int(xs):
  n = len(xs)
  k = to_binary(xs)
  if k == 0:
    return as_binary_string(k+1, n)
  elif k == 2**n - 1:
    return as_binary_string(k-1, n)
  else:
    return random.choice([as_binary_string(k-1, n), as_binary_string(k+1, n)])

For the neighbourhood induced by the Hamming distance, we can select a position inside the list and "flip" the corresponding bit:

In [9]:
def neigh_hamming(xs):
  n = len(xs)
  candidates = []
  pos = random.randint(0, n-1)
  return xs[0:pos] + [1 - xs[pos]] + xs[pos+1:n]

We can now define Hill Climbing similarly to random search _except_ for the fact that the nexxt solution to consider is selected in among the neighbours of the current best solution:

In [10]:
def hill_climbing(fit, neigh, start, n_iter = 100):
  best = start
  best_fit = fit(start)
  for i in range(0, n_iter):
    x = neigh(best)
    x_fit = fit(x)
    if x_fit >= best_fit:
      best = x
      best_fit = x_fit
  return best

We can now try Hill Climbing on the OneMax problem with both the "integer" neighbourhood and the one induced by the Hamming distance:

In [11]:
starting_point = random.choices([0,1], k = 30)
print(starting_point)
print(hill_climbing(onemax, neigh_int, starting_point))
print(hill_climbing(onemax, neigh_hamming, starting_point));

[0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]


## Simulated Annealing

We can now define the simulated annealing. In addition to what we have seen in the pseudocode, we also keep track of the best solution found so far, which is the one we return.

In [12]:
def simulated_annealing(fit, neigh, start, schedule, n_iter = 100):
  current = start
  current_fit = fit(start)
  best = current
  best_fit = current_fit
  T = schedule(0, n_iter)
  for i in range(0, n_iter):
    x = neigh(best)
    x_fit = fit(x)
    if x_fit >= current_fit:
      current = x
      current_fit = x_fit
      if x_fit >= best_fit:
        best = x
        best_fit = x_fit
    elif random.random() <= math.e**((x_fit - current_fit)/T):
      current = x
      current_fit = x_fit
    T = schedule(i, n_iter)
  return best

A possible schedule simply reduces the temperature with the number of iterations:

In [13]:
def schedule(i, n_iter):
  return n_iter/(i+1)

We can now check if something improves for the OneMax problem:

In [14]:
starting_point = random.choices([0,1], k = 20)
print(starting_point)
print(simulated_annealing(onemax, neigh_int, starting_point, schedule, n_iter=1000))
print(simulated_annealing(onemax, neigh_hamming, starting_point, schedule, n_iter = 1000))

[0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## Genetic Algorithms

We start by defining the tournament selection. Notice that ```max``` on a python list of tuples return the maximum one in lexicographic order:

In [15]:
def tournament_selection(pop, fit, k):
  tournament = random.choices(pop, k=k)
  selected = max([(fit(x), x) for x in tournament]) # (fitness, individual)
  return selected[1]

We define the one point crossover given two parents:

In [16]:
def one_point_crossover(x, y):
  n = len(x)
  k = random.randint(0,n-1)
  of1 = x[0:k] + y[k:n]
  of2 = y[0:k] + x[k:n]
  return of1, of2

In [17]:
one_point_crossover([1,1,1,1], [0,0,0,0])

([1, 1, 0, 0], [0, 0, 1, 1])

We can now define the bit-flip mutation with a given probability $p_\text{mut}$:

In [18]:
def bit_flip_mutation(x, p_m):
  def flip(b):
    if random.random() < p_m:
      return 1 - b
    else:
      return b
  
  return [flip(b) for b in x]

In [19]:
bit_flip_mutation([0,1,1,1,0], 0.2)

[0, 1, 1, 1, 0]

One essential step is the initial generation of the population. This can be done by generating uniformly at random ```pop_size``` individuals of $n$ bits:

In [20]:
def init_population(pop_size, n):
  return [random.choices([0,1], k = n) for _ in range(0, pop_size)]

We also add a function to return the best indvidual in the population and its fitness:

In [21]:
def get_best(pop, fit):
  return max([(fit(x), x) for x in pop]) # (best_finess, individual)

A single generation is done by performing three steps:
- selection (with tournament selection)
- crossover (with one-point crossover)
- mutation (with bit-flip mutation)

In [22]:
def generation(pop, fit, t_size, p_m):
  pop_size = len(pop)
  selected = [tournament_selection(pop, fit, t_size) for _ in range(0,pop_size)]
  offsprings = [one_point_crossover(x, y) for x,y in zip(selected[0::2], selected[1::2])] # (x0, x1), (x2, 3), ...
  offsprings = [ind for pair in offsprings for ind in pair]
  return list(map(lambda x: bit_flip_mutation(x, p_m), offsprings))

A GA simply perform a generational cycle a predefined number of times:

In [23]:
def GA(pop_size, n, fit, t_size = 4, n_gen = 10):
  p_m = 1/n
  pop = init_population(pop_size, n)
  for i in range(0, n_gen):
    print(get_best(pop, fit))
    pop = generation(pop, fit, t_size, p_m)
  return get_best(pop, fit)

In [24]:
GA(20, 20, onemax, n_gen=100)

(14, [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1])
(15, [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1])
(15, [1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0])
(17, [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1])
(17, [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1])
(18, [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])
(19, [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(19, [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(19, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1])
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

### Elitism

Now we can force the population to always contain the best individual found so far

In [25]:
from functools import reduce

def generation_elitist(pop, fit, t_size, p_m, elitism=True):
  best_fit, best = reduce(max, [(fit(x), x) for x in pop])
  pop_size = len(pop)
  selected = [tournament_selection(pop, fit, t_size) for _ in range(0,pop_size)]
  offsprings = [one_point_crossover(x, y) for x,y in zip(selected[0::2], selected[1::2])]
  offsprings = [ind for pair in offsprings for ind in pair]
  mutated_offsprings = list(map(lambda x: bit_flip_mutation(x, p_m), offsprings))
  best_fit_new, _ = reduce(max, [(fit(x), x) for x in mutated_offsprings])
  if best_fit_new < best_fit:
    # we remove one individual and replace it with the best
    mutated_offsprings[0] = best 
  return mutated_offsprings

def GA(pop_size, n, fit, t_size = 4, n_gen = 10, elitism = False):
  p_m = 1/n
  pop = init_population(pop_size, n)
  for i in range(0, n_gen):
    print(get_best(pop, fit))
    pop = generation_elitist(pop, fit, t_size, p_m, elitism)
  return get_best(pop, fit)

In [26]:
GA(10, 20, onemax, n_gen=20, elitism=True)


(15, [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1])
(16, [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1])
(16, [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1])
(16, [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1])
(17, [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1])
(17, [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1])
(17, [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1])
(18, [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])
(18, [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])
(18, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1])
(18, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

We now take a look at the knapsack problem

In [27]:
objects_value = [3, 5, 1, 9, 1, 4, 8, 2, 4, 2]
objects_weights = [8, 3, 1, 4, 1, 3, 2, 5, 7, 1]
max_capacity = 17

def knapsack_fitness(ind):
    tot_value = 0
    tot_weight = 0
    for i, choice in enumerate(ind):
        if choice == 1:
            tot_value += objects_value[i]
            tot_weight += objects_weights[i]
    if tot_weight > max_capacity:
        return -100
    else:
        return tot_value

In [28]:
random_sample = lambda: random.choices([0,1], k=10)
sol = random_search(knapsack_fitness, random_sample)
print(knapsack_fitness(sol), sol)

30 [0, 1, 1, 1, 1, 1, 1, 0, 0, 1]


In [29]:
starting_point = random.choices([0,1], k = 10)
print(starting_point)
sol1 = hill_climbing(knapsack_fitness, neigh_int, starting_point)
sol2 = hill_climbing(knapsack_fitness, neigh_hamming, starting_point)
print(knapsack_fitness(sol1), sol1)
print(knapsack_fitness(sol2), sol2)

[0, 1, 1, 0, 0, 0, 1, 0, 1, 0]
20 [0, 1, 1, 0, 0, 0, 1, 0, 1, 1]
21 [0, 1, 1, 0, 1, 0, 1, 0, 1, 1]


In [30]:
starting_point = random.choices([0,1], k = 10)
print(starting_point)
sol1 = simulated_annealing(knapsack_fitness, neigh_int, starting_point, schedule)
sol2 = simulated_annealing(knapsack_fitness, neigh_hamming, starting_point, schedule)
print(knapsack_fitness(sol1), sol1)
print(knapsack_fitness(sol2), sol2)

[1, 1, 1, 1, 1, 0, 1, 0, 1, 1]
-100 [1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
23 [0, 1, 1, 1, 0, 1, 0, 1, 0, 1]


In [31]:
GA(10, 10, knapsack_fitness, n_gen=20, elitism=True)

(18, [0, 0, 1, 1, 0, 1, 0, 0, 1, 0])
(20, [0, 0, 1, 1, 0, 1, 0, 0, 1, 1])
(20, [0, 0, 1, 1, 0, 1, 0, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])


(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])